In [1]:
import numpy as np
import pysindy as ps
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from sklearn.linear_model import Lasso
from pysindy.differentiation import SmoothedFiniteDifference

# Initialize integrator keywords for solve_ivp to replicate the odeint defaults
integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

In [ ]:
def vf(t, x, p=[1]):
    return [-x[1], - p[0] * (1 - x[0] ** 2) * x[1] + x[0]]
# Generate measurement data
dt = .001

t_train = np.arange(0, 100, dt)
x0_train = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
t_train_span = (t_train[0], t_train[-1])


x_train = solve_ivp(fun = vf, t_span = t_train_span, y0 = x0_train, t_eval=t_train, **integrator_keywords).y.T
diff_method = SmoothedFiniteDifference(smoother_kws={'window_length': 5})
opt = ps.optimizers.STLSQ(threshold=0.01, alpha = 0.1, max_iter=2000, normalize_columns=True, fit_intercept=False)
lib = ps.PolynomialLibrary(degree=3).fit(x_train)
model = ps.SINDy(optimizer=opt, feature_library=lib, differentiation_method=diff_method)
model.fit(x_train, t=dt)
print('score: {}'.format(model.score(x_train, dt)))
model.print()